In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random
from sklearn import preprocessing

import gc
from scipy.stats import skew, boxcox

from scipy import sparse
from sklearn.metrics import log_loss
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

seed = 2017

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU,LeakyReLU,ELU,ParametricSoftplus,ThresholdedReLU,SReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.optimizers import SGD,Nadam
from keras.regularizers import WeightRegularizer, ActivityRegularizer,l2, activity_l2
from keras.utils.np_utils import to_categorical

Using Theano backend.


# Load Data

In [13]:
data_path = "../input/"
train_X = pd.read_csv(data_path + 'train_BM_0331.csv')
test_X = pd.read_csv(data_path + 'test_BM_0331.csv')

ntrain = train_X.shape[0]
sub_id = test_X.listing_id.astype('int32').values
# all_features = features_to_use + desc_sparse_cols + feat_sparse_cols
print train_X.shape, test_X.shape, train_y.shape

(49352, 412) (74659, 412) (49352,)


In [14]:
null_ind = test_X.num_loc_price_diff.isnull()
test_X['num_loc_price_diff'] = test_X['num_price'] - test_X['num_loc_median_price']
# test_X[null_ind][['num_loc_price_diff','num_price','num_loc_median_price']]

In [15]:
time_feature = pd.read_csv(data_path + 'listing_image_time.csv')
time_feature.columns = ['listing_id','time_stamp']
train_X = train_X.merge(time_feature,on='listing_id',how='left')
test_X = test_X.merge(time_feature,on='listing_id',how='left')

print train_X.shape
print test_X.shape

(49352, 413)
(74659, 413)


In [16]:
full_data = pd.concat([train_X,test_X])
print full_data.shape

(124011, 413)


In [17]:
full_data.columns.values

array(['building_id_mean_med', 'building_id_mean_high',
       'manager_id_mean_med', 'manager_id_mean_high', 'median_price_bed',
       'ratio_bed', 'compound', 'neg', 'neu', 'pos', 'street', 'avenue',
       'east', 'west', 'north', 'south', 'other_address', 'top_10_manager',
       'top_25_manager', 'top_5_manager', 'top_50_manager',
       'top_1_manager', 'top_2_manager', 'top_15_manager',
       'top_20_manager', 'top_30_manager', 'Zero_building_id',
       'top_10_building', 'top_25_building', 'top_5_building',
       'top_50_building', 'top_1_building', 'top_2_building',
       'top_15_building', 'top_20_building', 'top_30_building',
       'listing_id', 'num_latitude', 'num_longitude',
       'num_dist_from_center', 'num_OutlierAggregated', 'num_pos_density',
       'num_building_null', 'num_fbuilding', 'num_fmanager',
       'num_created_weekday', 'num_created_weekofyear', 'num_created_day',
       'num_created_month', 'num_created_hour', 'num_bathrooms',
       'num_bedrooms

In [18]:
feat_to_use = ['building_id_mean_med', 'building_id_mean_high',
       'manager_id_mean_med', 'manager_id_mean_high', 'median_price_bed',
       'ratio_bed', 'compound', 'neg', 'neu', 'pos', 
#                'street', 'avenue',
#        'east', 'west', 'north', 'south', 'other_address', 'top_10_manager',
#        'top_25_manager', 'top_5_manager', 'top_50_manager',
#        'top_1_manager', 'top_2_manager', 'top_15_manager',
#        'top_20_manager', 'top_30_manager', 'Zero_building_id',
#        'top_10_building', 'top_25_building', 'top_5_building',
#        'top_50_building', 'top_1_building', 'top_2_building',
#        'top_15_building', 'top_20_building', 'top_30_building',
       'listing_id', 'num_latitude', 'num_longitude',
       'num_dist_from_center', 'num_OutlierAggregated', 'num_pos_density',
       'num_building_null', 'num_fbuilding', 'num_fmanager',
       'num_created_weekday', 'num_created_weekofyear', 'num_created_day',
       'num_created_month', 'num_created_hour', 'num_bathrooms',
       'num_bedrooms', 'num_price', 'num_price_q', 'num_priceXroom',
       'num_even_bathrooms', 'num_features', 'num_photos',
       'num_desc_length', 'num_desc_length_null', 
#                'num_location_6_3',
#        'num_location_6_1', 'num_location_6_0', 'num_location_6_5',
#        'num_location_6_4', 'num_location_6_2', 'num_location_40_18',
#        'num_location_40_31', 'num_location_40_11', 'num_location_40_24',
#        'num_location_40_14', 'num_location_40_36', 'num_location_40_3',
#        'num_location_40_7', 'num_location_40_33', 'num_location_40_5',
#        'num_location_40_37', 'num_location_40_12', 'num_location_40_16',
#        'num_location_40_2', 'num_location_40_20', 'num_location_40_34',
#        'num_location_40_9', 'num_location_40_0', 'num_location_40_21',
#        'num_location_40_26', 'num_location_40_13', 'num_location_40_25',
#        'num_location_40_32', 'num_location_40_19', 'num_location_40_17',
#        'num_location_40_4', 'num_location_40_15', 'num_location_40_35',
#        'num_location_40_22', 'num_location_40_30', 'num_location_40_1',
#        'num_location_40_23', 'num_location_40_10', 'num_location_40_38',
#        'num_location_40_28', 'num_location_40_6', 'num_location_40_29',
#        'num_location_40_27', 'num_location_40_39', 'num_location_40_8',
#        'num_room_type_0', 'num_room_type_1', 'num_room_type_2',
#        'num_room_type_3', 'num_room_type_4', 'num_room_type_5',
#        'num_room_type_6', 'num_room_type_7', 'num_room_type_8',
#        'num_room_type_9', 'num_room_type_10', 'num_room_type_11',
#        'num_room_type_12', 'num_room_type_13', 'num_room_type_14',
#        'num_room_type_15', 'num_room_type_16', 'num_room_type_17',
#        'num_room_type_18', 'num_room_type_19', 
               'num_6_median_price',
       'num_6_price_ratio', 'num_6_price_diff', 'num_loc_median_price',
       'num_loc_price_ratio', 'num_loc_price_diff', 'num_loc_ratio',
       'num_loc_diff', 'hcc_pos_pred_1', 'hcc_pos_pred_2', 'building_id',
       'display_address', 'manager_id', 'street_address',
       'num_pricePerBed', 'num_pricePerBath', 'num_pricePerRoom',
       'num_bedPerBath', 'num_bedBathDiff', 'num_bedBathSum',
       'num_bedsPerc', 
#                'hcc_building_id_pred_1', 'hcc_building_id_pred_2',
#        'hcc_manager_id_pred_1', 'hcc_manager_id_pred_2',
#        'feature_1_month_free', 'feature_24/7_concierge',
#        'feature_24/7_doorman', 'feature_24/7_doorman_concierge',
#        'feature_actual_apt._photos', 'feature_air_conditioning',
#        'feature_all_pets_ok', 'feature_all_utilities_included',
#        'feature_assigned-parking-space', 'feature_attended_lobby',
#        'feature_backyard', 'feature_balcony', 'feature_basement_storage',
#        'feature_basketball_court', 'feature_bike_room',
#        'feature_bike_storage', 'feature_billiards_room',
#        'feature_billiards_table_and_wet_bar', 'feature_brand_new',
#        'feature_breakfast_bar', 'feature_bright', 'feature_brownstone',
#        'feature_building-common-outdoor-space', 'feature_business_center',
#        'feature_cable/satellite_tv', 'feature_cable_ready',
#        'feature_call/text_abraham_caro_@_917-373-0862',
#        'feature_cats_allowed', 'feature_central_a/c', 'feature_central_ac',
#        'feature_central_air', 'feature_chefs_kitchen',
#        "feature_children's_playroom", 'feature_childrens_playroom',
#        'feature_cinema_room', 'feature_city_view',
#        'feature_close_to_subway', 'feature_closets_galore!',
#        'feature_club_sun_deck_has_spectacular_city_and_river_views',
#        'feature_cold_storage', 'feature_common_backyard',
#        'feature_common_garden', 'feature_common_outdoor_space',
#        'feature_common_parking/garage', 'feature_common_roof_deck',
#        'feature_common_storage', 'feature_common_terrace',
#        'feature_community_recreation_facilities',
#        'feature_complimentary_sunday_brunch', 'feature_concierge',
#        'feature_concierge_service', 'feature_condo_finishes',
#        'feature_courtyard', 'feature_crown_moldings', 'feature_deck',
#        'feature_deco_brick_wall', 'feature_decorative_fireplace',
#        'feature_dining_room', 'feature_dishwasher', 'feature_dogs_allowed',
#        'feature_doorman', 'feature_dry_cleaning_service',
#        'feature_dryer_in_unit', 'feature_duplex', 'feature_duplex_lounge',
#        'feature_eat-in_kitchen', 'feature_eat_in_kitchen',
#        'feature_elegant_glass-enclosed_private_lounge_with_magnificent_river_views',
#        'feature_elevator', 'feature_exclusive',
#        'feature_exercise/yoga_studio', 'feature_exposed_brick',
#        'feature_extra_room', 'feature_fireplace', 'feature_fireplaces',
#        'feature_fitness_center', 'feature_fitness_room', 'feature_flex-2',
#        'feature_flex-3', 'feature_free_wifi_in_club_lounge',
#        'feature_ft_doorman', 'feature_full-time_doorman',
#        'feature_full_service_garage',
#        'feature_fully-equipped_club_fitness_center',
#        'feature_fully__equipped', 'feature_furnished', 'feature_game_room',
#        'feature_garage', 'feature_garbage_disposal', 'feature_garden',
#        'feature_garden/patio', 'feature_granite_countertops',
#        'feature_granite_kitchen', 'feature_green_building',
#        'feature_guarantors_accepted', 'feature_gut_renovated',
#        'feature_gym', 'feature_gym/fitness', 'feature_gym_in_building',
#        'feature_hardwood', 'feature_hardwood_floors',
#        'feature_health_club', 'feature_hi_rise',
#        'feature_high-speed_internet', 'feature_high_ceiling',
#        'feature_high_ceilings', 'feature_high_speed_internet',
#        'feature_highrise', 'feature_housekeeping_service',
#        'feature_in-unit_washer/dryer', 'feature_indoor_pool',
#        'feature_intercom', 'feature_jacuzzi', 'feature_large_living_room',
#        'feature_laundry', 'feature_laundry_&_housekeeping',
#        'feature_laundry_in_building', 'feature_laundry_in_unit',
#        'feature_laundry_on_every_floor', 'feature_laundry_on_floor',
#        'feature_laundry_room', 'feature_light', 'feature_live-in_super',
#        'feature_live-in_superintendent', 'feature_live/work',
#        'feature_live_in_super', 'feature_loft', 'feature_lounge',
#        'feature_lounge_room', 'feature_lowrise', 'feature_luxury_building',
#        'feature_magnificent_venetian-style', 'feature_mail_room',
#        'feature_marble_bath', 'feature_marble_bathroom',
#        'feature_media_room', 'feature_media_screening_room',
#        'feature_microwave', 'feature_midrise', 'feature_multi-level',
#        'feature_new_construction', 'feature_newly_renovated',
#        'feature_no_fee', 'feature_no_pets', 'feature_on-site_atm_machine',
#        'feature_on-site_attended_garage', 'feature_on-site_garage',
#        'feature_on-site_laundry', 'feature_on-site_parking',
#        'feature_on-site_parking_available', 'feature_on-site_parking_lot',
#        'feature_on-site_super', 'feature_one_month_free',
#        'feature_outdoor_areas', 'feature_outdoor_entertainment_space',
#        'feature_outdoor_pool',
#        'feature_outdoor_roof_deck_overlooking_new_york_harbor_and_battery_park',
#        'feature_outdoor_space', 'feature_package_room', 'feature_parking',
#        'feature_parking_available', 'feature_parking_space',
#        'feature_part-time_doorman', 'feature_party_room', 'feature_patio',
#        'feature_penthouse', 'feature_pet_friendly', 'feature_pets',
#        'feature_pets_allowed', 'feature_pets_on_approval',
#        'feature_playroom', 'feature_playroom/nursery', 'feature_pool',
#        'feature_post-war', 'feature_post_war', 'feature_pre-war',
#        'feature_pre_war', 'feature_prewar', 'feature_private-balcony',
#        'feature_private-outdoor-space', 'feature_private_backyard',
#        'feature_private_balcony', 'feature_private_deck',
#        'feature_private_garden',
#        'feature_private_laundry_room_on_every_floor',
#        'feature_private_outdoor_space', 'feature_private_parking',
#        'feature_private_roof_deck', 'feature_private_roofdeck',
#        'feature_private_terrace', 'feature_publicoutdoor',
#        'feature_queen_size_bedrooms', 'feature_queen_sized_rooms',
#        'feature_reduced_fee', 'feature_renovated',
#        'feature_renovated_kitchen', 'feature_residents_garden',
#        'feature_residents_lounge', 'feature_roof-deck',
#        'feature_roof_access', 'feature_roof_deck',
#        'feature_roof_deck_with_grills', 'feature_roofdeck',
#        'feature_rooftop_deck', 'feature_rooftop_terrace',
#        'feature_s/s_appliances', 'feature_sauna', 'feature_screening_room',
#        'feature_separate_kitchen', 'feature_shared_backyard',
#        'feature_shared_garden', 'feature_shares_ok',
#        'feature_short_term_allowed', 'feature_simplex', 'feature_skylight',
#        'feature_skylight_atrium', 'feature_southern_exposure',
#        'feature_spa_services', 'feature_ss_appliances',
#        'feature_stainless_steel', 'feature_stainless_steel_appliances',
#        'feature_state-of-the-art_fitness_center', 'feature_storage',
#        'feature_storage_available', 'feature_storage_facilities_available',
#        'feature_storage_room', 'feature_sublet', 'feature_subway',
#        'feature_sundeck', 'feature_swimming_pool', 'feature_tenant_lounge',
#        'feature_terrace', 'feature_terraces_/_balconies',
#        'feature_tons_of_natural_light', 'feature_valet',
#        'feature_valet_parking', 'feature_valet_service',
#        'feature_valet_services',
#        'feature_valet_services_including_dry_cleaning',
#        'feature_video_intercom', 'feature_view', 'feature_virtual_doorman',
#        'feature_virtual_tour', 'feature_walk-in_closet', 'feature_walk-up',
#        'feature_walk_in_closet', 'feature_walk_in_closet(s)',
#        'feature_washer/dryer', 'feature_washer/dryer_hookup',
#        'feature_washer/dryer_in-unit', 'feature_washer/dryer_in_building',
#        'feature_washer/dryer_in_unit', 'feature_washer_&_dryer',
#        'feature_washer_in_unit', 'feature_wheelchair_access',
#        'feature_wheelchair_ramp', 'feature_wifi', 'feature_wifi_access',
#        'feature_wood-burning_fireplace', 'feature_yard',
#        'feature_yoga_classes', 
               'time_stamp']

In [19]:
# full_data = full_data.fillna(0)

for col in feat_to_use:
    full_data.loc[:,col] = (full_data[col]-full_data[col].mean())/full_data[col].std()
train_df_nn = full_data[:ntrain]
test_df_nn = full_data[ntrain:]

train_df_nn = sparse.csr_matrix(train_df_nn)
test_df_nn = sparse.csr_matrix(test_df_nn)


print train_df_nn.shape
print test_df_nn.shape

(49352, 413)
(74659, 413)


In [20]:
full_data.isnull().values.any()

False

In [21]:
def batch_generator(X, y, batch_size, shuffle):
    number_of_batches = np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    if shuffle:
        np.random.shuffle(sample_index)
    while True:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[batch_index,:].toarray()
        y_batch = y[batch_index]
        counter += 1
        yield X_batch, y_batch
        if (counter == number_of_batches):
            if shuffle:
                np.random.shuffle(sample_index)
            counter = 0

def batch_generatorp(X, batch_size, shuffle):
    number_of_batches = X.shape[0] / np.ceil(X.shape[0]/batch_size)
    counter = 0
    sample_index = np.arange(X.shape[0])
    while True:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]
        X_batch = X[batch_index, :].toarray()
        counter += 1
        yield X_batch
        if (counter == number_of_batches):
            counter = 0

In [22]:
train_y = np.ravel(pd.read_csv(data_path + 'labels_BrandenMurray.csv'))
# train_y = to_categorical(train_y)

# Low

In [23]:
y_low =[]
for i in range(train_X.shape[0]):
    y_low.append(1 if train_y[i] == 0 else 0)
    
y_low = np.array(y_low)  
print np.sum(y_low)
y_low = to_categorical(y_low)

34284


In [25]:


def nn_model(params):
    model = Sequential()
    init = 'glorot_normal'
    
    model.add(Dense(params['input_size'], # number of input units: needs to be tuned
                    input_dim = params['input_dim'], # fixed length: number of columns of X
                    init=init,
                   ))
    model.add(Activation('sigmoid'))
    model.add(PReLU()) # activation function
    model.add(BatchNormalization()) # normalization
    model.add(Dropout(params['input_drop_out'])) #dropout rate. needs to be tuned
        
    model.add(Dense(params['hidden_size'],
                    init=init)) # number of hidden1 units. needs to be tuned.
    model.add(Activation('sigmoid'))
    model.add(PReLU())
    model.add(BatchNormalization())    
    model.add(Dropout(params['hidden_drop_out'])) #dropout rate. needs to be tuned
    
#     model.add(Dense(20,init=init)) # number of hidden2 units. needs to be tuned.
#     model.add(Activation('sigmoid'))
#     model.add(PReLU())
#     model.add(BatchNormalization())    
#     model.add(Dropout(0.5)) #dropout rate. needs to be tuned
    
    model.add(Dense(2,
                    init = init,
                    activation = 'softmax')) # 1 for regression 
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'adam' # optimizer. you may want to try different ones
                 )
    return(model)



def nn_blend_data(parameters, train_x, train_y, test_x, fold, early_stopping_rounds=0, batch_size=128):
    N_params = len(parameters)
    print ("Blend %d estimators for %d folds" % (len(parameters), fold))
    skf = KFold(n_splits=fold,random_state=seed)
    N_class = train_y.shape[1]
    
    train_blend_x = np.zeros((train_x.shape[0], N_class*N_params))
    test_blend_x = np.zeros((test_x.shape[0], N_class*N_params))
    scores = np.zeros ((fold,N_params))
    best_rounds = np.zeros ((fold, N_params))
    

    
    for j, nn_params in enumerate(parameters):
        print ("Model %d: %s" %(j+1, nn_params))
        test_blend_x_j = np.zeros((test_x.shape[0], N_class*fold))
        
        for i, (train_index, val_index) in enumerate(skf.split(train_x)):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x[train_index]
            train_y_fold = train_y[train_index]
            val_x_fold = train_x[val_index]
            val_y_fold = train_y[val_index]
            

            model = nn_model(nn_params)
#             print (model)
            fit= model.fit_generator(generator=batch_generator(train_x_fold, train_y_fold, 128, True),
                                     nb_epoch=60,
                                     samples_per_epoch=train_x_fold.shape[0],
                                     validation_data=(val_x_fold.todense(), val_y_fold),
                                     verbose = 0,
                                     callbacks=[ModelCheckpoint(filepath="weights.hdf5", 
                                                                monitor='val_loss', 
                                                                verbose=0, save_best_only=True)]
                                    )

            best_round=len(fit.epoch)-early_stopping_rounds-1
            best_rounds[i,j]=best_round
            print ("best round %d" % (best_round))
            
            model.load_weights("weights.hdf5")
            # Compile model (required to make predictions)
            model.compile(loss = 'categorical_crossentropy',optimizer = 'adam' )
            
            # print (mean_absolute_error(np.exp(y_val)-200, pred_y))
            val_y_predict_fold = model.predict_proba(x=val_x_fold.toarray(),verbose=0)
            score = log_loss(val_y_fold, val_y_predict_fold)
            print ("Score: ", score)
            scores[i,j]=score   
            train_blend_x[val_index, (j*N_class):(j+1)*N_class] = val_y_predict_fold
            
            model.load_weights("weights.hdf5")
            # Compile model (required to make predictions)
            model.compile(loss = 'categorical_crossentropy',optimizer = 'adam' )            
            test_blend_x_j[:,(i*N_class):(i+1)*N_class] = model.predict_proba(x=test_x.toarray(),verbose=0)
            print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))            
            
        test_blend_x[:,(j*N_class):(j+1)*N_class] = \
                np.stack([test_blend_x_j[:,range(0,N_class*fold,N_class)].mean(1),
                          test_blend_x_j[:,range(1,N_class*fold,N_class)].mean(1)]).T
            
        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x, scores,best_rounds)

In [29]:
nn_parameters = [
    { 'input_size' :100 ,
     'input_dim' : train_X.shape[1],
     'input_drop_out' : 0.4 ,
     'hidden_size' : 30 ,
     'hidden_drop_out' :0.4}
]

(train_blend_x_low, test_blend_x_low, blend_scores,best_round) = nn_blend_data(nn_parameters, train_df_nn, y_low, test_df_nn,
                                                         5,
                                                         5)

Blend 1 estimators for 5 folds
Model 1: {'input_size': 100, 'input_drop_out': 0.4, 'hidden_drop_out': 0.4, 'hidden_size': 30, 'input_dim': 413}
Model 1 fold 1


/home/xujin/AI/anaconda2/lib/python2.7/site-packages/keras/engine/training.py:1480: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


best round 54
('Score: ', 0.37421376879215185)
Model 1 fold 1 fitting finished in 235.699s
Model 1 fold 2
best round 54
('Score: ', 0.36841183297683294)
Model 1 fold 2 fitting finished in 236.860s
Model 1 fold 3
best round 54
('Score: ', 0.38790396247247827)
Model 1 fold 3 fitting finished in 213.752s
Model 1 fold 4
best round 54
('Score: ', 0.40022028249241065)
Model 1 fold 4 fitting finished in 213.394s
Model 1 fold 5
best round 54
('Score: ', 0.40787803853078575)
Model 1 fold 5 fitting finished in 233.945s
Score for model 1 is 0.387726
Score for blended models is 0.387726


In [30]:

now = datetime.now()

name_train_blend = '../output/train_blend_Keras_last_ovr_low_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend = '../output/test_blend_Keras_last_ovr_low_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'



print (np.mean(blend_scores,axis=0))
print (np.mean(best_round,axis=0))
np.savetxt(name_train_blend,train_blend_x_low, delimiter=",")
np.savetxt(name_test_blend,test_blend_x_low, delimiter=",")

[ 0.38772558]
[ 54.]


# medium

In [31]:
y_medium =[]
for i in range(train_X.shape[0]):
    y_medium.append(1 if train_y[i] == 1 else 0)
    
y_medium = np.array(y_medium)  
print np.sum(y_medium)
y_medium = to_categorical(y_medium)

11229


In [32]:
nn_parameters = [
    { 'input_size' :100 ,
     'input_dim' : train_X.shape[1],
     'input_drop_out' : 0.4 ,
     'hidden_size' : 30 ,
     'hidden_drop_out' :0.4}

]

(train_blend_x_medium , test_blend_x_medium , blend_scores,best_round) = nn_blend_data(nn_parameters,train_df_nn, 
                                                                                       y_medium , 
                                                                                       test_df_nn,
                                                         5,
                                                         5)

Blend 1 estimators for 5 folds
Model 1: {'input_size': 100, 'input_drop_out': 0.4, 'hidden_drop_out': 0.4, 'hidden_size': 30, 'input_dim': 413}
Model 1 fold 1
best round 54
('Score: ', 0.41109054657353966)
Model 1 fold 1 fitting finished in 227.868s
Model 1 fold 2
best round 54
('Score: ', 0.41109962720917409)
Model 1 fold 2 fitting finished in 213.886s
Model 1 fold 3
best round 54
('Score: ', 0.42121349797050278)
Model 1 fold 3 fitting finished in 211.931s
Model 1 fold 4
best round 54
('Score: ', 0.43516122966743753)
Model 1 fold 4 fitting finished in 212.816s
Model 1 fold 5
best round 54
('Score: ', 0.43336667688648389)
Model 1 fold 5 fitting finished in 212.341s
Score for model 1 is 0.422386
Score for blended models is 0.422386


In [33]:
now = datetime.now()

name_train_blend = '../output/train_blend_Keras_last_ovr_medium_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend = '../output/test_blend_Keras_last_ovr_medium_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'



print (np.mean(blend_scores,axis=0))
print (np.mean(best_round,axis=0))
np.savetxt(name_train_blend,train_blend_x_medium, delimiter=",")
np.savetxt(name_test_blend,test_blend_x_medium, delimiter=",")

[ 0.42238632]
[ 54.]


# High

In [34]:
y_high =[]
for i in range(train_X.shape[0]):
    y_high.append(1 if train_y[i] == 2 else 0)
    
y_high = np.array(y_high)  
print np.sum(y_high)
y_high = to_categorical(y_high)

3839


In [35]:
nn_parameters = [
    { 'input_size' :100 ,
     'input_dim' : train_X.shape[1],
     'input_drop_out' : 0.4 ,
     'hidden_size' : 30 ,
     'hidden_drop_out' :0.4}

]

(train_blend_x_hihg, test_blend_x_hihg, blend_scores,best_round) = nn_blend_data(nn_parameters,train_df_nn, 
                                                                                       y_high , 
                                                                                       test_df_nn,
                                                         5,
                                                         5)

Blend 1 estimators for 5 folds
Model 1: {'input_size': 100, 'input_drop_out': 0.4, 'hidden_drop_out': 0.4, 'hidden_size': 30, 'input_dim': 413}
Model 1 fold 1
best round 54
('Score: ', 0.17188711641413681)
Model 1 fold 1 fitting finished in 212.458s
Model 1 fold 2
best round 54
('Score: ', 0.17506999335706461)
Model 1 fold 2 fitting finished in 224.481s
Model 1 fold 3
best round 54
('Score: ', 0.18049783294262139)
Model 1 fold 3 fitting finished in 213.791s
Model 1 fold 4
best round 54
('Score: ', 0.18485708338559365)
Model 1 fold 4 fitting finished in 214.936s
Model 1 fold 5
best round 54
('Score: ', 0.1783687703980249)
Model 1 fold 5 fitting finished in 215.180s
Score for model 1 is 0.178136
Score for blended models is 0.178136


In [36]:
now = datetime.now()

name_train_blend = '../output/train_blend_Keras_last_ovr_high_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend = '../output/test_blend_Keras_last_ovr_high_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'



print (np.mean(blend_scores,axis=0))
print (np.mean(best_round,axis=0))
np.savetxt(name_train_blend,train_blend_x_hihg, delimiter=",")
np.savetxt(name_test_blend,test_blend_x_hihg, delimiter=",")

[ 0.17813616]
[ 54.]


# combine

In [41]:
train_blend_x = np.vstack([train_blend_x_low[:,1],train_blend_x_medium[:,1],train_blend_x_hihg[:,1]]).T
train_blend_x.shape

(49352, 3)

In [42]:
test_blend_x_mean = np.vstack([test_blend_x_low[:,1],test_blend_x_medium[:,1],test_blend_x_hihg[:,1]]).T
test_blend_x_mean.shape

(74659, 3)

In [43]:
name_train_blend = '../output/train_blend_Keras_ovr_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend_mean = '../output/test_blend_Keras_ovr_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'



# print (np.mean(blend_scores_xgb,axis=0))
# print (np.mean(best_rounds_xgb,axis=0))
np.savetxt(name_train_blend,train_blend_x, delimiter=",")
np.savetxt(name_test_blend_mean,test_blend_x_mean, delimiter=",")


In [45]:
# now = datetime.now()
sub_name = '../output/sub_Keras_mean_ovr_last_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

out_df = pd.DataFrame(test_blend_x_mean[:,:3])
out_df.columns = ["low", "medium", "high"]
out_df["listing_id"] = test_X.listing_id.values
out_df.to_csv(sub_name, index=False)